In [1]:
from rembg import remove
import os
import random
import json
import numpy as np
import cv2
from PIL import  Image, ImageDraw, ImageEnhance, ImageFilter
from pycocotools import mask
from skimage import measure
import json

In [3]:
#define paths
imagesObjectsPath = "imagesObjects/"
croppedPath  = "images_cropped/"
fg_folders = [
    ("coca/","cocacola" ),
    ("banana/","banana" )
]
bg_folder = "images_bg/"
output_folder = "dataset/"
fg_folders



[('coca/', 'cocacola'), ('banana/', 'banana')]

In [4]:
#remuevo Fondo verde
for folder in fg_folders:
    for filename in os.listdir(f"{imagesObjectsPath}{folder[0]}"):
        myImage = Image.open(f"{imagesObjectsPath}{folder[0]}{filename}")
        myCroppedImage=remove(myImage) 
        myCroppedImage.save(f"{imagesObjectsPath}{folder[0]}{filename}")
       

In [5]:
#recorto imagenes
for folder in fg_folders:
    for filename in os.listdir(f"{imagesObjectsPath}{folder[0]}"):
        try:
            myImage = Image.open(f"{imagesObjectsPath}{folder[0]}{filename}")
            black = Image.new('RGBA', myImage.size)
            myImage = Image.composite(myImage, black, myImage)
            myCroppedImage = myImage.crop(myImage.getbbox())
            myCroppedImage.save(f"{croppedPath}{folder[0]}{filename}")
        except:
            continue

In [6]:
objects_list = ["cocacola","banana"]
annotations_ID = {}
categories = []
for i, object in enumerate(objects_list):
    annotations_ID[object] = i
    categories.append({"id": i, "name": object})

print(annotations_ID)
print(categories)


{'cocacola': 0, 'banana': 1}
[{'id': 0, 'name': 'cocacola'}, {'id': 1, 'name': 'banana'}]


In [7]:
# Load the list of files in each of the three folders
fg_files = {}
for folder, category in fg_folders:
    fg_files[category] = os.listdir(f"{croppedPath}{folder}")

fg_files

{'cocacola': ['cocaCola_100.png',
  'cocaCola_101.png',
  'cocaCola_102.png',
  'cocaCola_103.png',
  'cocaCola_104.png',
  'cocaCola_105.png',
  'cocaCola_106.png',
  'cocaCola_107.png',
  'cocaCola_108.png',
  'cocaCola_109.png',
  'cocaCola_110.png',
  'cocaCola_111.png',
  'cocaCola_112.png',
  'cocaCola_113.png',
  'cocaCola_114.png',
  'cocaCola_115.png',
  'cocaCola_116.png',
  'cocaCola_117.png',
  'cocaCola_118.png',
  'cocaCola_119.png',
  'cocaCola_120.png',
  'cocaCola_121.png',
  'cocaCola_122.png',
  'cocaCola_123.png',
  'cocaCola_124.png',
  'cocaCola_125.png',
  'cocaCola_126.png',
  'cocaCola_127.png',
  'cocaCola_128.png',
  'cocaCola_129.png',
  'cocaCola_130.png',
  'cocaCola_131.png',
  'cocaCola_132.png',
  'cocaCola_133.png',
  'cocaCola_134.png',
  'cocaCola_135.png',
  'cocaCola_136.png',
  'cocaCola_137.png',
  'cocaCola_138.png',
  'cocaCola_139.png',
  'cocaCola_140.png',
  'cocaCola_141.png',
  'cocaCola_142.png',
  'cocaCola_143.png',
  'cocaCola_144.png'

In [8]:
# Declare two empty list that will help as storage for the images and annotations of the dataset
images=[]
annotations=[]
annotations2=[]
#Declare in 0 the img id and annotation id
img_id=int(0)
anno_id=int(0)

rescaling_min = 0.1
rescaling_max = 0.5

# Ratios at which these values will be modified
brightness_ratio = 0.05
saturation_ratio = 0.05
hue_ratio = 0.02

In [9]:
for j in range(500):
    #select hramdomly how many objects will be in an image
    num_objects = random.randint(0, 3)
    print("number of objects",num_objects)
    # Select random foreground images from the three folders, with replacement
    fg_categories = random.choices(objects_list, k=num_objects)
    
    fg_files_selected = []
    for category in fg_categories:
        fg_files_selected.append([category,random.choice(fg_files[category])])
    print("seleccion",fg_files_selected)
    # Load the selected foreground images using Pillow
    fg_imgs = []
    for img in fg_files_selected:
        folder = (f"{croppedPath}{[f[0] for f in fg_folders if f[1] == img[0]][0]}")
        fg_imgs.append([img[0],Image.open(folder + img[1]),folder+img[1]])

    # Randomly resize and rotate the foreground images using Pillow's transform module
    # img[0] is category, img[1] is image, img[2] is path
    for img in fg_imgs:
        fg_img=img[1]
        angle = random.randint(-5, 5)
        scale = random.uniform(rescaling_min, rescaling_max)
        fg_img = fg_img.rotate(angle, resample=Image.BICUBIC, expand=True)
        fg_img = fg_img.resize((int(fg_img.width * scale), int(fg_img.height * scale)))
        fg_img = ImageEnhance.Brightness(fg_img).enhance(random.uniform(1.0, 1.5))
        fg_img = ImageEnhance.Contrast(fg_img).enhance(random.uniform(0.9, 1.1))
        fg_img = ImageEnhance.Color(fg_img).enhance(random.uniform(0.7, 1.3))
        fg_img = fg_img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.0, 0.5)))


        img[1] = fg_img

    # Load the background image using Pillow
    bg_files = os.listdir(bg_folder)
    bg_file = random.choice(bg_files)
    bg_img = Image.open(bg_folder + bg_file)

    # Define the maximum overlap as a percentage
    max_overlap_pct = 10



    # Define an array to keep track of occupied areas
    occupied = np.zeros((bg_img.height, bg_img.width))

    for img in fg_imgs:
        fg_img=img[1]

        # Calculate the maximum overlap area
        max_overlap_area = (fg_img.width * fg_img.height)

        seg_img = img[1]


        # Convert the image to a NumPy array
        img_arr = np.array(seg_img)
        # Create a binary mask of the non-transparent pixels
        mask = img_arr[:, :, 3] != 0

        # Convert the mask to a COCO format segmentation
        segmentation = []
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i, j]:
                    segmentation.append(j)
                    segmentation.append(i)
        segmentation = [segmentation]

        # Calculate the area of the segmentation
        area = 0
        for i in range(len(segmentation[0]) // 2):
            x1 = segmentation[0][2 * i]
            y1 = segmentation[0][2 * i + 1]
            x2 = segmentation[0][(2 * i + 2) % len(segmentation[0])]
            y2 = segmentation[0][(2 * i + 3) % len(segmentation[0])]
            area += x1 * y2 - x2 * y1
        area = abs(area) / 2
        
        # Draw the segmentation onto a copy of the original image
        #image_copy = image.copy()
        #cv2.fillPoly(image_copy, aux_segmentation, color=(0, 255, 0))

        # Display the image with segmentation overlay
        #cv2.imshow('Image with Segmentation', image_copy)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        # Calculate the maximum allowed position for the top-left corner
        max_x = bg_img.width - fg_img.width
        max_y = bg_img.height - fg_img.height
        area = fg_img.width * fg_img.height

        # Generate a random location until an unoccupied area is found that meets the overlap limit
        total_area = bg_img.width * bg_img.height
        overlap_area = total_area
        
        while overlap_area / area > max_overlap_pct / 100:
            x = random.randint(0, max_x)
            y = random.randint(0, max_y)

            # Calculate the overlap area
            overlap_area = np.sum(occupied[y:y+fg_img.height, x:x+fg_img.width])

            # Check if the area is unoccupied and the overlap limit is not exceeded
            if (max_overlap_area) >= overlap_area/10:
                break
            if i==10:
                continue
        
        for i in range(0, len(segmentation[0])):
            if i % 2:
                segmentation[0][i]=int(segmentation[0][i]+y)
            else :
                segmentation[0][i]=int(segmentation[0][i]+x)
        # Update the occupied array
        occupied[y:y+fg_img.height, x:x+fg_img.width] = 1
        bg_img.paste(fg_img, (x, y), fg_img)
        annotations2.append({"id": anno_id,"image_id": img_id,"category_id": annotations_ID[img[0]],"bbox": [x, y, fg_img.width, fg_img.height],"segmentation": segmentation,"area": area,"iscrowd": 0})
        annotations.append({"id": anno_id,"image_id": img_id,"category_id": annotations_ID[img[0]],"bbox": [x, y, fg_img.width, fg_img.height],"segmentation": [],"area": fg_img.height*fg_img.width,"iscrowd": 0})
        anno_id=anno_id+1
        #draw = ImageDraw.Draw(bg_img)
        #fdraw.rectangle((x, y, x+fg_img.width, y+fg_img.height), outline='red', width=3)
    bg_img.save("dataset/"+str(img_id)+".jpg", quality=100)
    images.append({"id": img_id, "file_name": str(img_id)+".jpg","height": bg_img.height,"width": bg_img.width})
    img_id=img_id+1

number of objects 1
seleccion [['cocacola', 'cocaCola_188.png']]
number of objects 2
seleccion [['cocacola', 'cocaCola_162.png'], ['cocacola', 'cocaCola_168.png']]
number of objects 0
seleccion []
number of objects 3
seleccion [['banana', 'banana_31.png'], ['cocacola', 'cocaCola_122.png'], ['cocacola', 'cocaCola_122.png']]
number of objects 2
seleccion [['banana', 'banana_99.png'], ['cocacola', 'cocaCola_142.png']]
number of objects 0
seleccion []
number of objects 0
seleccion []
number of objects 3
seleccion [['cocacola', 'cocaCola_145.png'], ['cocacola', 'cocaCola_157.png'], ['cocacola', 'cocaCola_163.png']]
number of objects 0
seleccion []
number of objects 3
seleccion [['banana', 'banana_61.png'], ['banana', 'banana_97.png'], ['banana', 'banana_36.png']]
number of objects 2
seleccion [['cocacola', 'cocaCola_166.png'], ['cocacola', 'cocaCola_154.png']]
number of objects 3
seleccion [['cocacola', 'cocaCola_149.png'], ['banana', 'banana_87.png'], ['cocacola', 'cocaCola_185.png']]
numb

In [10]:
# Define the COCO dictionary
coco_dict = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}
coco_dict2 = {
    "images": images,
    "annotations": annotations2,
    "categories": categories
}

coco_dict2
# Convert the dictionary to a JSON string
json_string = json.dumps(coco_dict2)

# Write the JSON string to a file
with open("annotations_val_seg.json", 'w') as f:
    f.write(json_string)

coco_dict
# Convert the dictionary to a JSON string
json_string = json.dumps(coco_dict)

# Write the JSON string to a file
with open("annotations_val.json", 'w') as f:
    f.write(json_string)